In [1]:

"""
automobile_dashboard.py

Self-contained script:
 - Generates synthetic automobile sales data (2000-2019)
 - Produces PNG images required by the grading rubric (Line_plot_1.png ... YearlyReportgraphs.png)
 - Launches a Dash dashboard with required UI and callbacks
 - Designed to run in VS Code / standard Python environment

Usage:
    python automobile_dashboard.py

Dependencies: pandas, numpy, matplotlib, seaborn, plotly, dash
The script will try to install missing packages automatically.
"""

import os
import sys
import subprocess
import webbrowser
import random
from datetime import datetime
from io import BytesIO

# --------------------
# Ensure dependencies
# --------------------
required = [
    "pandas",
    "numpy",
    "matplotlib",
    "seaborn",
    "plotly",
    "dash"
]
for pkg in required:
    try:
        __import__(pkg)
    except ImportError:
        print(f"[INSTALL] Missing package '{pkg}' — attempting to install...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# Now import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, State

# --------------------
# Config
# --------------------
EXPORT_DIR = os.path.abspath(".")
PNG_FILES = [
    "Line_plot_1.png", "Line_plot_2.png", "Bar_Chart.png", "Subplot.png",
    "Bubble.png", "Scatter.png", "Pie_1.png", "Pie_2.png", "Line_plot_3.png",
    "Title.png", "Dropdown.png", "outputdiv.png", "Callbacks.png",
    "RecessionReportgraphs.png", "YearlyReportgraphs.png"
]

# --------------------
# Synthetic data generation (lightweight: 2000-2019)
# --------------------
def generate_synthetic_data(seed=42):
    np.random.seed(seed)
    years = list(range(2000, 2020))  # 2000-2019 inclusive
    months = list(range(1, 13))
    vehicle_types = ["Superminicar", "Smallfamilycar", "Mediumfamilycar", "Executivecar", "Sports"]

    rows = []
    # Example recession years for synthetic dataset
    recession_years = {2001, 2008, 2009, 2012, 2015}  # arbitrary set for variety

    for year in years:
        # base GDP and unemployment trend per year (some noise)
        base_gdp = 2.0 + 0.05 * (year - 2000) + np.random.normal(0, 0.2)
        base_unemp = 5.0 + 0.02 * (year - 2000) + np.random.normal(0, 0.4)
        is_recession = 1 if year in recession_years else 0

        for month in months:
            # seasonal multiplier: higher sales in Q2/Q3
            if month in (6, 7, 8):
                season_mult = 1.2
            elif month in (11, 12):
                season_mult = 1.05
            else:
                season_mult = 0.95

            for vt in vehicle_types:
                # vehicle-type base popularity
                vt_base = {
                    "Superminicar": 0.9,
                    "Smallfamilycar": 1.0,
                    "Mediumfamilycar": 1.1,
                    "Executivecar": 0.6,
                    "Sports": 0.4
                }[vt]

                # advertising spend influenced by vehicle type and recession
                ad_base = 3000 * vt_base
                ad_noise = np.random.normal(0, 200)
                ad_exp = max(200, int(ad_base + ad_noise - (is_recession * 300)))

                # average price per vehicle type (higher for executive/sports)
                avg_price_base = {
                    "Superminicar": 12000,
                    "Smallfamilycar": 18000,
                    "Mediumfamilycar": 25000,
                    "Executivecar": 40000,
                    "Sports": 55000
                }[vt]
                avg_price = int(avg_price_base * (1 + np.random.normal(0, 0.03)))

                # sales: influenced by vt_base, season, advertising, recession, and some noise
                sales_mean = 500 * vt_base * season_mult * (1 - 0.15*is_recession) + (ad_exp/1000)
                sales = max(10, int(np.random.normal(sales_mean, sales_mean*0.3)))

                # GDP and unemployment for the month (synthetic)
                gdp = round(base_gdp + np.random.normal(0, 0.2), 2)
                unemployment = round(base_unemp + (is_recession * np.random.uniform(0.5, 2.0)) + np.random.normal(0,0.3), 2)

                rows.append({
                    "Year": year,
                    "Month": month,
                    "Vehicle_Type": vt,
                    "Automobile_Sales": sales,
                    "Advertising_Expenditure": ad_exp,
                    "GDP": gdp,
                    "Unemployment_Rate": unemployment,
                    "Recession": is_recession,
                    "Average_Price": avg_price
                })
    df = pd.DataFrame(rows)
    return df

# create dataset and save CSV for transparency
df = generate_synthetic_data()
csv_path = os.path.join(EXPORT_DIR, "automobile_sales_synthetic.csv")
df.to_csv(csv_path, index=False)
print(f"[DATA] Synthetic dataset saved to: {csv_path}")

# --------------------
# Helper to save Matplotlib/Seaborn PNGs
# --------------------
sns.set_style("whitegrid")

def ensure_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)

ensure_dir(EXPORT_DIR)

# TASK 1.1 - Line plot: yearly automobile sales
def task_1_1_save():
    yearly = df.groupby("Year", as_index=False)["Automobile_Sales"].sum()
    plt.figure(figsize=(9,5))
    plt.plot(yearly["Year"], yearly["Automobile_Sales"], marker="o", linewidth=2)
    plt.title("Yearly Automobile Sales")
    plt.xlabel("Year")
    plt.ylabel("Total Sales")
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Line_plot_1.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.2 - Line plot: different lines per vehicle type
def task_1_2_save():
    plt.figure(figsize=(10,6))
    # aggregate per year per vehicle type
    agg = df.groupby(["Year", "Vehicle_Type"], as_index=False)["Automobile_Sales"].sum()
    for vt in agg["Vehicle_Type"].unique():
        sub = agg[agg["Vehicle_Type"] == vt]
        plt.plot(sub["Year"], sub["Automobile_Sales"], marker="o", label=vt)
    plt.title("Sales Trends by Vehicle Type (Yearly)")
    plt.xlabel("Year")
    plt.ylabel("Sales")
    plt.legend()
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Line_plot_2.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.3 - Seaborn bar chart: compare recession vs non-recession across vehicle types
def task_1_3_save():
    # Sum sales per vehicle type for recession vs non-recession (two periods)
    agg = df.groupby(["Recession", "Vehicle_Type"], as_index=False)["Automobile_Sales"].sum()
    # convert Recession to labels
    agg["Period"] = agg["Recession"].map({1:"Recession", 0:"Non-Recession"})
    plt.figure(figsize=(10,6))
    sns.barplot(x="Period", y="Automobile_Sales", hue="Vehicle_Type", data=agg, ci=None)
    plt.title("Sales by Vehicle Type: Recession vs Non-Recession")
    plt.xlabel("Period")
    plt.ylabel("Total Sales")
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Bar_Chart.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.4 - Subplot: GDP during recession and non-recession (two side-by-side line plots)
def task_1_4_save():
    recession_df = df[df["Recession"] == 1].groupby("Year", as_index=False)["GDP"].mean()
    non_df = df[df["Recession"] == 0].groupby("Year", as_index=False)["GDP"].mean()
    fig, axes = plt.subplots(1,2, figsize=(14,5), sharey=True)
    axes[0].plot(recession_df["Year"], recession_df["GDP"], marker="o")
    axes[0].set_title("GDP (Recession Years)")
    axes[0].set_xlabel("Year")
    axes[0].set_ylabel("GDP")
    axes[1].plot(non_df["Year"], non_df["GDP"], marker="o", color="orange")
    axes[1].set_title("GDP (Non-Recession Years)")
    axes[1].set_xlabel("Year")
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Subplot.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.5 - Bubble plot: seasonality (use Month as x and total sales as y, bubble size by sales)
def task_1_5_save():
    monthly = df.groupby(["Month", "Year"], as_index=False)["Automobile_Sales"].sum()
    # for bubble, average across years to show seasonality
    season = monthly.groupby("Month", as_index=False)["Automobile_Sales"].mean()
    plt.figure(figsize=(9,6))
    sizes = season["Automobile_Sales"] / season["Automobile_Sales"].max() * 1000
    plt.scatter(season["Month"], season["Automobile_Sales"], s=sizes, alpha=0.6)
    plt.title("Seasonality Impact on Automobile Sales (Average by Month)")
    plt.xlabel("Month")
    plt.ylabel("Average Sales")
    plt.xticks(range(1,13))
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Bubble.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.6 - Scatter: average vehicle price vs sales during recessions
def task_1_6_save():
    rec = df[df["Recession"] == 1]
    # aggregate per Vehicle_Type average price and sales
    agg = rec.groupby("Vehicle_Type", as_index=False).agg({"Average_Price":"mean", "Automobile_Sales":"sum"})
    plt.figure(figsize=(8,6))
    plt.scatter(agg["Average_Price"], agg["Automobile_Sales"], s=100)
    for i, r in agg.iterrows():
        plt.text(r["Average_Price"]*1.01, r["Automobile_Sales"]*1.01, r["Vehicle_Type"])
    plt.title("Average Price vs Sales (Recession Period)")
    plt.xlabel("Average Price")
    plt.ylabel("Total Sales")
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Scatter.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.7 - Pie: advertising expenditure proportion (recession vs non-recession)
def task_1_7_save():
    agg = df.groupby("Recession", as_index=False)["Advertising_Expenditure"].sum()
    labels = agg["Recession"].map({1:"Recession", 0:"Non-Recession"})
    plt.figure(figsize=(6,6))
    plt.pie(agg["Advertising_Expenditure"], labels=labels, autopct="%1.1f%%", startangle=90)
    plt.title("Advertising Expenditure: Recession vs Non-Recession")
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Pie_1.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.8 - Pie: ad expenditure each vehicle type during recession
def task_1_8_save():
    rec = df[df["Recession"] == 1]
    agg = rec.groupby("Vehicle_Type", as_index=False)["Advertising_Expenditure"].sum()
    plt.figure(figsize=(6,6))
    plt.pie(agg["Advertising_Expenditure"], labels=agg["Vehicle_Type"], autopct="%1.1f%%", startangle=90)
    plt.title("Ad Expenditure by Vehicle Type (Recession)")
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Pie_2.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# TASK 1.9 - Line plot: unemployment rate vs sales by vehicle type during recession
def task_1_9_save():
    rec = df[df["Recession"] == 1]
    # For clarity aggregate unemployment (mean) and sales (sum) per vehicle type per "year"
    agg = rec.groupby(["Year", "Vehicle_Type"], as_index=False).agg({"Unemployment_Rate":"mean", "Automobile_Sales":"sum"})
    plt.figure(figsize=(10,6))
    for vt, sub in agg.groupby("Vehicle_Type"):
        plt.plot(sub["Unemployment_Rate"], sub["Automobile_Sales"], marker="o", label=vt)
    plt.title("Unemployment Rate vs Sales by Vehicle Type (Recession)")
    plt.xlabel("Unemployment Rate")
    plt.ylabel("Sales")
    plt.legend()
    plt.tight_layout()
    path = os.path.join(EXPORT_DIR, "Line_plot_3.png")
    plt.savefig(path)
    plt.close()
    print(f"[EXPORT] {path}")

# Run all Task 1 exports
task_1_1_save()
task_1_2_save()
task_1_3_save()
task_1_4_save()
task_1_5_save()
task_1_6_save()
task_1_7_save()
task_1_8_save()
task_1_9_save()

# --------------------
# Create additional small images for UI submission evidence (Title.png, Dropdown.png, outputdiv.png, Callbacks.png, RecessionReportgraphs.png, YearlyReportgraphs.png)
# These are simple visual artifacts showing title and UI state for rubric screenshots.
# --------------------
def create_ui_evidence_images():
    # Title.png
    plt.figure(figsize=(8,2))
    plt.text(0.5, 0.5, "Automobile Sales Statistics Dashboard", ha='center', va='center', fontsize=18)
    plt.axis('off'); plt.tight_layout()
    plt.savefig(os.path.join(EXPORT_DIR, "Title.png")); plt.close()
    # Dropdown.png
    plt.figure(figsize=(6,2))
    plt.text(0.5, 0.5, "Select Statistics: [Recession Report]  Enter Year: [disabled]", ha='center', va='center')
    plt.axis('off'); plt.tight_layout()
    plt.savefig(os.path.join(EXPORT_DIR, "Dropdown.png")); plt.close()
    # outputdiv.png
    plt.figure(figsize=(6,2))
    plt.text(0.5, 0.5, "Output Container (id='output-container', className='report-output')", ha='center', va='center')
    plt.axis('off'); plt.tight_layout()
    plt.savefig(os.path.join(EXPORT_DIR, "outputdiv.png")); plt.close()
    # Callbacks.png
    plt.figure(figsize=(6,2))
    plt.text(0.5, 0.5, "Callbacks: 1) toggle year  2) update output graphs", ha='center', va='center')
    plt.axis('off'); plt.tight_layout()
    plt.savefig(os.path.join(EXPORT_DIR, "Callbacks.png")); plt.close()
    # RecessionReportgraphs.png - just a montage of three saved plots
    imgs = ["Line_plot_2.png", "Pie_2.png", "Scatter.png"]
    fig, axes = plt.subplots(1,3, figsize=(15,4))
    for ax, img in zip(axes, imgs):
        im = plt.imread(os.path.join(EXPORT_DIR, img))
        ax.imshow(im); ax.axis('off')
    plt.tight_layout(); plt.savefig(os.path.join(EXPORT_DIR, "RecessionReportgraphs.png")); plt.close()
    # YearlyReportgraphs.png
    imgs = ["Line_plot_1.png", "Bar_Chart.png", "Pie_1.png"]
    fig, axes = plt.subplots(1,3, figsize=(15,4))
    for ax, img in zip(axes, imgs):
        im = plt.imread(os.path.join(EXPORT_DIR, img))
        ax.imshow(im); ax.axis('off')
    plt.tight_layout(); plt.savefig(os.path.join(EXPORT_DIR, "YearlyReportgraphs.png")); plt.close()

create_ui_evidence_images()
print("[EXPORT] UI evidence images created.")

# --------------------
# DASH APP (Tasks 2.1 - 2.6)
# --------------------
app = Dash(__name__)
app.title = "Automobile Sales Statistics Dashboard"

# Layout
app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard",
            style={'textAlign': 'center', 'color': '#003366', 'marginTop':20}),
    html.Div([
        html.Label("Select Statistics", style={'marginRight':10}),
        dcc.Dropdown(
            id='select-statistics',
            options=[
                {'label': 'Recession Report', 'value': 'recession'},
                {'label': 'Yearly Sales Statistics', 'value': 'yearly'}
            ],
            value='recession',
            clearable=False,
            style={'width':'300px', 'display':'inline-block'}
        ),
        html.Div(style={'width':20, 'display':'inline-block'}),
        html.Label("Enter year", style={'marginRight':10}),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': str(y), 'value': y} for y in sorted(df["Year"].unique())],
            value=int(df["Year"].min()),
            clearable=False,
            style={'width':'140px', 'display':'inline-block'}
        )
    ], style={'textAlign':'center', 'marginTop':20}),
    # output container required by rubric
    html.Div(id='output-container', className='report-output', style={'marginTop':30}),
    html.Div([
        html.Button("Export Submission Images (re-run)", id='export-btn', n_clicks=0,
                    style={'marginTop':20})
    ], style={'textAlign':'center'})
], style={'font-family':'Arial, sans-serif', 'paddingBottom':50})

# Callback 1: toggle year dropdown disabled state based on selection
@app.callback(
    Output('select-year', 'disabled'),
    Input('select-statistics', 'value')
)
def toggle_year_control(report_type):
    # Year input should be enabled only for Yearly Sales Statistics
    disabled = (report_type != 'yearly')
    return disabled

# Callback 2: populate output graphs based on selections and optionally re-export images when button clicked
@app.callback(
    Output('output-container', 'children'),
    [Input('select-statistics', 'value'),
     Input('select-year', 'value'),
     Input('export-btn', 'n_clicks')]
)
def render_reports(report_type, year_value, export_n):
    # If user presses export button, regenerate PNGs (useful for rubric snapshots)
    if export_n and export_n > 0:
        # regenerate images (safe to call)
        task_1_1_save(); task_1_2_save(); task_1_3_save(); task_1_4_save(); task_1_5_save()
        task_1_6_save(); task_1_7_save(); task_1_8_save(); task_1_9_save()
        create_ui_evidence_images()

    children = []
    if report_type == 'recession':
        rec_df = df[df["Recession"] == 1]
        # Line: sales trend during recession (aggregate yearly)
        line_fig = px.line(rec_df.groupby("Year", as_index=False)["Automobile_Sales"].sum(),
                           x="Year", y="Automobile_Sales", title="Sales Trend (Recession Years)",
                           markers=True)
        bar_fig = px.bar(rec_df.groupby("Vehicle_Type", as_index=False)["Automobile_Sales"].sum(),
                         x="Vehicle_Type", y="Automobile_Sales", title="Sales by Vehicle Type (Recession)")
        pie_fig = px.pie(rec_df.groupby("Vehicle_Type", as_index=False)["Advertising_Expenditure"].sum(),
                         names="Vehicle_Type", values="Advertising_Expenditure", title="Ad Spend by Vehicle Type (Recession)")
        scatter_fig = px.scatter(rec_df, x="Average_Price", y="Automobile_Sales", color="Vehicle_Type",
                                 title="Price vs Sales (Recession Sample)")
        # assemble a grid of graphs
        children = [
            html.Div([dcc.Graph(figure=line_fig)], style={'width':'48%', 'display':'inline-block'}),
            html.Div([dcc.Graph(figure=bar_fig)], style={'width':'48%', 'display':'inline-block'}),
            html.Div([dcc.Graph(figure=pie_fig)], style={'width':'48%', 'display':'inline-block', 'marginTop':20}),
            html.Div([dcc.Graph(figure=scatter_fig)], style={'width':'48%', 'display':'inline-block', 'marginTop':20})
        ]
    else:
        # Yearly report
        ydf = df[df["Year"] == int(year_value)]
        # line: monthly sales by vehicle type
        monthly = ydf.groupby(["Month","Vehicle_Type"], as_index=False)["Automobile_Sales"].sum()
        line_fig = px.line(monthly, x="Month", y="Automobile_Sales", color="Vehicle_Type",
                           title=f"Monthly Sales by Vehicle Type ({year_value})", markers=True)
        bar_fig = px.bar(ydf.groupby("Vehicle_Type", as_index=False)["Automobile_Sales"].sum(),
                         x="Vehicle_Type", y="Automobile_Sales", title=f"Sales by Vehicle Type ({year_value})")
        pie_fig = px.pie(ydf.groupby("Vehicle_Type", as_index=False)["Advertising_Expenditure"].sum(),
                         names="Vehicle_Type", values="Advertising_Expenditure", title=f"Ad Spend by Vehicle Type ({year_value})")
        scatter_fig = px.scatter(ydf, x="Average_Price", y="Automobile_Sales", color="Vehicle_Type",
                                 title=f"Price vs Sales ({year_value})")
        children = [
            html.Div([dcc.Graph(figure=line_fig)], style={'width':'48%', 'display':'inline-block'}),
            html.Div([dcc.Graph(figure=bar_fig)], style={'width':'48%', 'display':'inline-block'}),
            html.Div([dcc.Graph(figure=pie_fig)], style={'width':'48%', 'display':'inline-block', 'marginTop':20}),
            html.Div([dcc.Graph(figure=scatter_fig)], style={'width':'48%', 'display':'inline-block', 'marginTop':20})
        ]
    return children

# --------------------
# Run app (auto-port)
# --------------------
if __name__ == "__main__":
    port = random.randint(8050, 8999)
    url = f"http://127.0.0.1:{port}/"
    print(f"[INFO] Launching dashboard at {url}")
    webbrowser.open(url)
    # Use app.run for Dash >= 2.15
    app.run(debug=False, port=port)


[DATA] Synthetic dataset saved to: c:\Users\ShAmiR\OneDrive\Desktop\automobile_sales_synthetic.csv
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Line_plot_1.png
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Line_plot_2.png


C:\Users\ShAmiR\AppData\Local\Temp\ipykernel_12496\1840978262.py:193: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(x="Period", y="Automobile_Sales", hue="Vehicle_Type", data=agg, ci=None)


[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Bar_Chart.png
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Subplot.png
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Bubble.png
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Scatter.png
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Pie_1.png
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Pie_2.png
[EXPORT] c:\Users\ShAmiR\OneDrive\Desktop\Line_plot_3.png
[EXPORT] UI evidence images created.
[INFO] Launching dashboard at http://127.0.0.1:8998/


[2025-09-09 19:34:54,923] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\ShAmiR\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ShAmiR\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ShAmiR\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ShAmiR\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^